In [1]:
pip install networkx matplotlib pandas


Note: you may need to restart the kernel to use updated packages.


In [33]:
import os
print("Current Working Directory:", os.getcwd())


Current Working Directory: C:\Users\aditi\DAA R


In [ ]:
import os
import subprocess
import tkinter as tk
from tkinter import ttk, messagebox
from tkinter.scrolledtext import ScrolledText
import matplotlib.pyplot as plt

# List of cities
cities = [
    "Dehradun", "Haridwar", "Rishikesh", "Mussoorie", "Nainital", "Haldwani",
    "Rudrapur", "Kashipur", "Roorkee", "Almora", "Pithoragarh", "Chamoli",
    "Tehri", "Uttarkashi", "Joshimath", "Bageshwar", "Doiwala", "Kichha",
    "Sitarganj", "Khatima", "Tanakpur", "Lohaghat", "Ranikhet", "Kotdwara", "New Tehri"
]

class RouteRoverApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Route Rover - Uttarakhand")
        self.geometry("650x500")
        self.resizable(False, False)
        
        self.create_widgets()
        
    def create_widgets(self):
        # Title
        title = ttk.Label(self, text="Route Rover - Uttarakhand", font=("Arial", 18, "bold"))
        title.grid(row=0, column=0, columnspan=3, pady=10)

        # Source city dropdown
        ttk.Label(self, text="Select Source City:").grid(row=1, column=0, sticky="w", padx=10)
        self.source_var = tk.StringVar()
        self.source_cb = ttk.Combobox(self, textvariable=self.source_var, values=cities, state="readonly")
        self.source_cb.grid(row=1, column=1, padx=10, sticky="ew")
        self.source_cb.bind("<<ComboboxSelected>>", self.validate_selection)

        # Destination city dropdown
        ttk.Label(self, text="Select Destination City:").grid(row=2, column=0, sticky="w", padx=10, pady=5)
        self.dest_var = tk.StringVar()
        self.dest_cb = ttk.Combobox(self, textvariable=self.dest_var, values=cities, state="readonly")
        self.dest_cb.grid(row=2, column=1, padx=10, sticky="ew")
        self.dest_cb.bind("<<ComboboxSelected>>", self.validate_selection)

        # Find Route button
        self.find_btn = ttk.Button(self, text="Find Shortest Route", command=self.find_route, state="disabled")
        self.find_btn.grid(row=3, column=0, columnspan=2, pady=10)

        # Clear button
        clear_btn = ttk.Button(self, text="Clear", command=self.clear_all)
        clear_btn.grid(row=3, column=2, pady=10, padx=10, sticky="e")

        # Output text box with scrollbar
        self.output_text = ScrolledText(self, height=15, width=75, wrap=tk.WORD)
        self.output_text.grid(row=4, column=0, columnspan=3, padx=10, pady=5)

        # Status bar
        self.status_var = tk.StringVar(value="Select source and destination cities.")
        status_bar = ttk.Label(self, textvariable=self.status_var, relief=tk.SUNKEN, anchor="w")
        status_bar.grid(row=5, column=0, columnspan=3, sticky="ew")

        # Plot graph button
        plot_btn = ttk.Button(self, text="Show Top 3 Cities Graph", command=self.show_top3_graph)
        plot_btn.grid(row=6, column=0, columnspan=3, pady=10)

        # Configure grid weights for better resizing (optional)
        self.columnconfigure(1, weight=1)

    def validate_selection(self, event=None):
        source = self.source_var.get()
        dest = self.dest_var.get()
        if source and dest and source != dest:
            self.find_btn.config(state="normal")
            self.status_var.set("Ready to find route.")
        else:
            self.find_btn.config(state="disabled")
            self.status_var.set("Select valid source and destination cities (different).")

    def clear_all(self):
        self.source_var.set("")
        self.dest_var.set("")
        self.output_text.delete(1.0, tk.END)
        self.find_btn.config(state="disabled")
        self.status_var.set("Selections cleared. Please select cities.")

    def find_route(self):
        source = self.source_var.get().strip()
        destination = self.dest_var.get().strip()

        if source == "" or destination == "":
            messagebox.showerror("Error", "Please select both source and destination cities.")
            return
        if source == destination:
            messagebox.showerror("Error", "Source and destination cannot be the same.")
            return

        base_path = os.getcwd()
        input_path = os.path.join(base_path, "input.txt")
        output_path = os.path.join(base_path, "output.txt")

        # Write input file
        try:
            with open(input_path, "w") as f:
                f.write(f"{source} {destination}\n")
        except Exception as e:
            messagebox.showerror("File Error", f"Failed to write input file:\n{e}")
            return

        # Run the graph.exe process
        try:
            completed_process = subprocess.run(
                ["graph.exe"],
                cwd=base_path,
                capture_output=True,
                text=True,
                timeout=10,
                check=True,
            )
        except subprocess.TimeoutExpired:
            messagebox.showerror("Timeout", "Running graph.exe timed out.")
            return
        except subprocess.CalledProcessError as e:
            messagebox.showerror("Execution Failed", f"graph.exe failed:\n{e.stderr}")
            return
        except Exception as e:
            messagebox.showerror("Execution Failed", f"Error running graph.exe:\n{e}")
            return

        # Read output file and display
        if os.path.exists(output_path):
            try:
                with open(output_path, "r") as f:
                    result = f.read()
            except Exception as e:
                messagebox.showerror("File Error", f"Failed to read output file:\n{e}")
                return
            self.output_text.delete(1.0, tk.END)
            self.output_text.insert(tk.END, result)
            self.status_var.set("Route found successfully.")
        else:
            self.output_text.delete(1.0, tk.END)
            self.output_text.insert(tk.END, "No output generated by graph.exe.")
            self.status_var.set("Output file not found.")

    def show_top3_graph(self):
        # Dummy data for top 3 cities - you can replace with real metrics if available
        top3_cities = ["Dehradun", "Haridwar", "Rishikesh"]
        values = [120, 95, 75]  # e.g., number of connections or visits or some metric

        plt.figure(figsize=(7,4))
        bars = plt.bar(top3_cities, values, color=["#1f77b4", "#ff7f0e", "#2ca02c"])
        plt.title("Top 3 Cities - Example Metrics")
        plt.ylabel("Metric Value")
        plt.xlabel("City")
        plt.grid(axis='y', linestyle='--', alpha=0.7)

        # Add value labels on bars
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2, height + 2, f'{height}', ha='center')

        plt.tight_layout()
        plt.show()

if __name__ == "__main__":
    app = RouteRoverApp()
    app.mainloop()
